# Pseudo-labeling Flow
---
0. Finish Augmentation & Create folder for testing (don't forgot to delete after train)
1. Read a file, Augmented Them, Put them in the Run it into the ConvNext Network, Get the Label -> Write it to file
2. Do different Augmented Here, 



99. output -> Save CSV that show -> Original Label, ConvNext Label, Swin Label, True Label.



In [1]:
import os
import albumentations as A
from PIL import Image
import torch
import numpy as np
from transformers import AutoModelForImageClassification, AutoImageProcessor
from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
import shutil

In [7]:
def apply_weak_augment(image):
    transform = A.Compose([
        
        A.HorizontalFlip(p=0.5), #both
        A.RandomGamma(p=0.5), #weak
        A.RandomBrightnessContrast(p=0.5), #weak
        A.AdvancedBlur(p=0.5), #weak
        A.Rotate (limit=10, p=0.5),
    ])
    image = Image.open(image_path).convert('RGB')
    augmented_image = transform(image=np.array(image))['image']
    augmented_image = Image.fromarray(augmented_image)
    return augmented_image
    
def apply_strong_augment(image):
    transform = A.Compose([
        A.GridDropout (ratio=0.5, unit_size_min=2, unit_size_max=3, holes_number_x=None, holes_number_y=None, shift_x=0, shift_y=0, random_offset=False, fill_value=0, mask_fill_value=None, always_apply=False, p=0.2),
#         A.CLAHE (clip_limit=4.0, tile_grid_size=(8, 8), always_apply=True, p=1), # Heavy Contrast Limited Adaptive Histogram Equalization
        A.Affine(), #heavy (shear + rotate)
#         A.RandomSunFlare(src_radius=200), #heavy
        A.CoarseDropout (max_holes=8, max_height=8, max_width=8, min_holes=None, min_height=None, min_width=None, fill_value=0, mask_fill_value=None, always_apply=False, p=0.5),#heavy
        A.RandomCrop(450,450), #80% of image size, probably both
#         A.GaussNoise (var_limit=(10.0, 50.0), mean=0, per_channel=True, always_apply=False, p=0.5),
        
        A.RandomFog(), # heavy
    ])
    image = Image.open(image_path).convert('RGB')
    augmented_image = transform(image=np.array(image))['image']
    augmented_image = Image.fromarray(augmented_image)
    return augmented_image
    

    

def display_image(image_path):
    try:
        img = Image.open(image_path)
        img.show()
    except Exception as e:
        print(f"Error opening image: {e}")

def append_row(df, img_name, conv_results, swin_result):
    new_row = {
        "img_name": img_name,
        "conv_results": conv_results,
        "swin_result": swin_result
    }
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    
    
def print_rows_with_same_values(df):
    counter = 0
    for index, row in df.iterrows():
        if row['conv_results'] == row['swin_results']:
            #print(row['img_name'])
            counter = counter +1
            
    return counter

def print_rows_with_both_correct_label(df):
    counter = 0
    for index, row in df.iterrows():
        if row['original_label'] == row['swin_results'] and row['original_label'] == row['conv_results']:
            #print(row['img_name'])
            counter = counter +1
            
    return counter
            

def print_rows_with_correct_label(df):
    counter = 0
    for index, row in df.iterrows():
        if row['original_label'] == row['swin_results'] or row['original_label'] == row['conv_results']:
            #print(row['img_name'])
            counter = counter +1
            
    return counter
            
def print_rows_with_correct_swin(df):
    counter = 0
    for index, row in df.iterrows():
        if row['original_label'] == row['swin_results']:
            #print(row['img_name'])
            counter = counter +1
            
    return counter

def print_rows_with_correct_conv(df):
    counter = 0
    for index, row in df.iterrows():
        if row['original_label'] == row['conv_results']:
            #print(row['img_name'])
            counter = counter +1
            
    return counter
    
def print_rows_with_correct_ensemble(df):
    counter = 0
    for index, row in df.iterrows():
        if row['conv_results'] == row['original_label'] and row['swin_results'] == row['original_label'] :
            #print(row['img_name'])
            counter = counter +1
            
    return counter

    
def create_pseudo_labeling_folder(df):
    correct_label = []
    for index, row in df.iterrows():
        if row['conv_results'] == row['original_label'] and row['conv_results'] == row['original_label'] :
            #print(row['img_name'])
            counter = counter +1
            
    return counter


def copy_and_move_image(image_path, subfolder_name,destination_path):
    try:
        # Check if the image_path exists
        if not os.path.exists(image_path):
            print(f"Error: Source image '{image_path}' does not exist.")
            return

        # Check if the destination directory exists, if not, create it
        if not os.path.exists(destination_path):
            os.makedirs(destination_path)
        
        
        
        # Get the filename from the image_path
        filename = os.path.basename(image_path)

        # Create the full destination path including the filename
        destination_file_path = os.path.join(destination_path,subfolder_name)
        if not os.path.exists(destination_file_path):
            os.makedirs(destination_file_path)
        destination_file_path = os.path.join(destination_file_path, filename)
        
        
        
        
        # Copy the image to the destination directory
        shutil.copy(image_path, destination_file_path)

        #print(f"Image copied and moved successfully to '{destination_file_path}'.")
    except Exception as e:
        print(f"Error: {e}")
        
def combine_folders(source_folders, destination_folder):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Iterate through each source folder
    for source_folder in source_folders:
        # Traverse through the source folder and its subdirectories to find all files and folders
        for root, dirs, files in os.walk(source_folder):
            # Create corresponding subdirectories in the destination folder
            dest_subfolder = os.path.join(destination_folder, os.path.relpath(root, source_folder))
            if not os.path.exists(dest_subfolder):
                os.makedirs(dest_subfolder)

            # Copy files to the destination folder
            for file in files:
                src_path = os.path.join(root, file)
                dest_path = os.path.join(dest_subfolder, file)
                shutil.copy(src_path, dest_path)

        
def show_augment(image):
    transform = A.Compose([
        
        #rotate
        #shear
        A.HorizontalFlip(p=0.5), #both
        #A.RandomFog(), # weak
        #A.RandomGamma(p=0.5), #weak
        #A.RandomBrightnessContrast(p=0.5), #weak
        #A.AdvancedBlur(p=0.5), #weak
        #A.Rotate (limit=10, p=0.5),
        
        #A.GridDropout (ratio=0.5, unit_size_min=2, unit_size_max=3, holes_number_x=None, holes_number_y=None, shift_x=0, shift_y=0, random_offset=False, fill_value=0, mask_fill_value=None, always_apply=False, p=0.2),
        #A.CLAHE (clip_limit=4.0, tile_grid_size=(8, 8), always_apply=True, p=1), # Heavy Contrast Limited Adaptive Histogram Equalization
        #A.Affine(), heavy (shear + rotate)
        #A.RandomRotate90(p=0.5) #heavy
        #A.RandomSunFlare(src_radius=200), #heavy
        #A.Cutout (num_holes=8, max_h_size=8, max_w_size=8, fill_value=0, p=0.5),#heavy
        #A.RandomCrop(450,450), #80% of image size, probably both
        #A.GaussNoise (var_limit=(10.0, 50.0), mean=0, per_channel=True, always_apply=False, p=0.5),
    ])
    image = Image.open(image).convert('RGB')
    augmented_image = transform(image=np.array(image))['image']
    augmented_image = Image.fromarray(augmented_image)
    augmented_image.show()
    

In [3]:
!pwd

/home/thanawit/twin-pseudo


In [20]:
image_formats = (".png", ".jpg", ".jpeg", ".gif", ".bmp")  # Add more supported formats if needed

result = pd.DataFrame(columns=["img_name", "conv_results", "swin_results","original_label"])

# 30% Test
# imagefolders = "/home/thanawit/twin-pseudo/cross_val_test/30%_CV/30%_Fold_0/unlabeled"
# #Swin Model
# swin_model_path = 'outputs_all/conv_output_30%_Fold_0/checkpoint-2600/'
# swin_model = AutoModelForImageClassification.from_pretrained(swin_model_path).to('cuda')
# swin_image_processor = AutoImageProcessor.from_pretrained(swin_model_path)

# #ConvNext Model
# conv_model_path = 'outputs_all/swin_output_30%_Fold_0/checkpoint-2600/'
# conv_model = AutoModelForImageClassification.from_pretrained(conv_model_path).to('cuda')
# conv_image_processor = AutoImageProcessor.from_pretrained(conv_model_path)



# 50% Test 
imagefolders = "/home/thanawit/twin-pseudo/cross_val_test/30%_CV/30%_Fold_0/unlabeled"
#imagefolders = "/home/thanawit/twin-pseudo/datasets/original"


#ConvNext Model // 6
conv_model_path = 'outputs_all/conv_output_30%_Fold_6/checkpoint-2600/'
conv_model = AutoModelForImageClassification.from_pretrained(conv_model_path).to('cuda')
conv_image_processor = AutoImageProcessor.from_pretrained(conv_model_path)



#Swin Model //6
swin_model_path = 'outputs_all/swin_combine_pseudo_30%_Fold_6/checkpoint-6100/'
swin_model = AutoModelForImageClassification.from_pretrained(swin_model_path).to('cuda')
swin_image_processor = AutoImageProcessor.from_pretrained(swin_model_path)


for root, subfolders, files in os.walk(imagefolders):
    for filename in files:
        if filename.lower().endswith(image_formats):
            image_path = os.path.join(root, filename)
            subfolder_name = os.path.basename(root)

            #print(f"Subfolder Name: {subfolder_name}")
            #print(image_path)
            img_strong = apply_weak_augment(image_path)
            #img = Image.open(image_path)
            
            #CONNVEXT PART
            
            #read model
            conv_inputs = conv_image_processor(images=img_strong,return_tensors="pt").to('cuda')
            #run img
            conv_output = conv_model(**conv_inputs)
            conv_logits = conv_output.logits
            conv_class = conv_logits.argmax(-1).item()
            conv_classname = conv_model.config.id2label[conv_class]
            
            #SWIN PART
            
            img_weak = apply_strong_augment(image_path)
            
            swin_inputs = swin_image_processor(images=img_weak,return_tensors="pt").to('cuda')
            
            swin_output = swin_model(**swin_inputs)
            swin_logits = swin_output.logits
            swin_class = swin_logits.argmax(-1).item()
            swin_classname = swin_model.config.id2label[swin_class]
            #get class
            new_row = {
                "img_name": os.path.basename(image_path),
                "conv_results": conv_classname,
                "swin_results": swin_classname,
                "original_label":subfolder_name
            }
            
            result = pd.concat([result, pd.DataFrame([new_row])],ignore_index=True)
            #save 
            #break
            #img.show()
            #print("=" * 30)
        #print(filename)

In [21]:
result

,img_name,conv_results,swin_results,original_label
0,73e1bb826f00.png,Typical,Atypical,Atypical
1,e393f9122068.png,Indeterminate,Typical,Atypical
2,b5916a232a5c.png,Indeterminate,Atypical,Atypical
3,8bf7d61dddb4.png,Atypical,Atypical,Atypical
4,093e5e69a2af.png,Atypical,Atypical,Atypical
...,...,...,...,...
3797,01bf68877dab.png,Negative,Negative,Negative
3798,7a4c7eca54a7.png,Typical,Typical,Negative
3799,055fc441c5e4.png,Negative,Negative,Negative
3800,f00ddc79378d.png,Negative,Negative,Negative


In [22]:
#counter = print_rows_with_correct_label(result)
print("Total Record",len(result))
print("Correct Result from Convnet",print_rows_with_correct_conv(result))
print("Correct Result from Swinn",print_rows_with_correct_swin(result))
print("Correct Result from Any Correct Label",print_rows_with_correct_label(result))
print("Correct Label From Both Framework",print_rows_with_correct_label(result))


print("Result where both are same ",print_rows_with_same_values(result))
print("Result where both are correct ",print_rows_with_both_correct_label(result))

Total Record 3802
Correct Result from Convnet 2923
Correct Result from Swinn 2451
Correct Result from Any Correct Label 3232
Correct Label From Both Framework 3232
Result where both are same  2496
Result where both are correct  2142


In [195]:
basefolder_name = (os.path.dirname(imagefolders))
fold_no = os.path.dirname(os.path.dirname(imagefolders))

for index, row in result.iterrows():
    if row['conv_results'] == row['original_label'] or row['swin_results'] == row['original_label'] :
        temp = os.path.join(imagefolders, row['original_label'])
        temp2 = os.path.join( temp,row['img_name'] )
        copy_and_move_image(temp2,row['original_label'] ,os.path.join(basefolder_name, 'pseudo_labeled'  ))
        
# combine_folders([os.path.join(basefolder_name,'pseudo_labeled')],
#                 os.path.join(basefolder_name,'combine_pseudo_labeled') )


train_folder = os.path.join(basefolder_name,'train')
pseudo_folder = os.path.join(basefolder_name,'pseudo_labeled')
source_folders = []
source_folders.append(train_folder)
source_folders.append(pseudo_folder)
#destination_folder = os.path.join(basefolder_name,'combine_pseudo_labeled')
# for the ensemble paper
#destination_folder = os.path.join(basefolder_name,'combine_pseudo_labeled')
destination_folder = 'cross_val_test/ensemble/30%/ensemble_strong_weak/'                           
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

#Iterate through each source folder
for source_folder in source_folders:
    # Traverse through the source folder and its subdirectories to find all files and folders
    for root, dirs, files in os.walk(source_folder):
        # Create corresponding subdirectories in the destination folder
        dest_subfolder = os.path.join(destination_folder, os.path.relpath(root, source_folder))
        if not os.path.exists(dest_subfolder):
            os.makedirs(dest_subfolder)

        # Copy files to the destination folder
        for file in files:
            src_path = os.path.join(root, file)
            dest_path = os.path.join(dest_subfolder, file)
            shutil.copy(src_path, dest_path)


In [173]:
'cross_val_test/ensemble/'

'cross_val_test/ensemble/'

In [213]:
basefolder_name = os.path.join(os.path.dirname(os.path.dirname(imagefolders)))

In [238]:
os.path.join(basefolder_name, 'pseudo_labeled')

'/home/thanawit/twin-pseudo/cross_val_test/30%_CV/30%_Fold_0/pseudo_labeled'

# CV Script

'/home/thanawit/twin-pseudo/cross_val_test/30%_CV'

In [15]:

for fold_counter in range(7,10):
    print(fold_counter)

7
8
9


In [265]:
swin_model_path

'outputs_all/conv_output_30%_Fold_fold_counter/checkpoint-2600/'

In [17]:
image_formats = (".png", ".jpg", ".jpeg", ".gif", ".bmp")  # Add more supported formats if needed

for fold_counter in range(6,10):
    result = pd.DataFrame(columns=["img_name", "conv_results", "swin_results","original_label"])

    imagefolders = "/home/thanawit/twin-pseudo/cross_val_test/50%_CV/50%_Fold_"+ str(fold_counter) +"/unlabeled"

    #Swin Model

    swin_model_path = 'outputs_all/conv_output_50%_Fold_'+ str(fold_counter) +  '/checkpoint-4300/'
    swin_model = AutoModelForImageClassification.from_pretrained(swin_model_path).to('cuda')
    swin_image_processor = AutoImageProcessor.from_pretrained(swin_model_path)

    #ConvNext Model
    conv_model_path = 'outputs_all/swin_output_50%_Fold_'+ str(fold_counter) +  '/checkpoint-4300/'
    conv_model = AutoModelForImageClassification.from_pretrained(conv_model_path).to('cuda')
    conv_image_processor = AutoImageProcessor.from_pretrained(conv_model_path)
    
    
    for root, subfolders, files in os.walk(imagefolders):
        for filename in files:
            if filename.lower().endswith(image_formats):
                image_path = os.path.join(root, filename)
                subfolder_name = os.path.basename(root)
                img_strong = apply_weak_augment(image_path)

                #CONNVEXT PART

                #read model
                conv_inputs = conv_image_processor(images=img_strong,return_tensors="pt").to('cuda')
                #run img
                conv_output = conv_model(**conv_inputs)
                conv_logits = conv_output.logits
                conv_class = conv_logits.argmax(-1).item()
                conv_classname = conv_model.config.id2label[conv_class]

                #SWIN PART

                img_weak = apply_weak_augment(image_path)

                swin_inputs = swin_image_processor(images=img_weak,return_tensors="pt").to('cuda')

                swin_output = swin_model(**swin_inputs)
                swin_logits = swin_output.logits
                swin_class = swin_logits.argmax(-1).item()
                swin_classname = swin_model.config.id2label[swin_class]
                #get class
                new_row = {
                    "img_name": os.path.basename(image_path),
                    "conv_results": conv_classname,
                    "swin_results": swin_classname,
                    "original_label":subfolder_name
                }

                result = pd.concat([result, pd.DataFrame([new_row])],ignore_index=True)
    #END FOR
    
    
    basefolder_name = (os.path.dirname(imagefolders))
    fold_no = os.path.dirname(os.path.dirname(imagefolders))

    for index, row in result.iterrows():
        if row['conv_results'] == row['original_label'] or row['swin_results'] == row['original_label'] :
            temp = os.path.join(imagefolders, row['original_label'])
            temp2 = os.path.join( temp,row['img_name'] )
            copy_and_move_image(temp2,row['original_label'] ,os.path.join(basefolder_name, 'pseudo_labeled'  ))

    # combine_folders([os.path.join(basefolder_name,'pseudo_labeled')],
    #                 os.path.join(basefolder_name,'combine_pseudo_labeled') )


    train_folder = os.path.join(basefolder_name,'train')
    pseudo_folder = os.path.join(basefolder_name,'pseudo_labeled')
    source_folders = []
    source_folders.append(train_folder)
    source_folders.append(pseudo_folder)
    destination_folder = os.path.join(basefolder_name,'combine_pseudo_labeled')

    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Iterate through each source folder
    for source_folder in source_folders:
        # Traverse through the source folder and its subdirectories to find all files and folders
        for root, dirs, files in os.walk(source_folder):
            # Create corresponding subdirectories in the destination folder
            dest_subfolder = os.path.join(destination_folder, os.path.relpath(root, source_folder))
            if not os.path.exists(dest_subfolder):
                os.makedirs(dest_subfolder)

            # Copy files to the destination folder
            for file in files:
                src_path = os.path.join(root, file)
                dest_path = os.path.join(dest_subfolder, file)
                shutil.copy(src_path, dest_path)
    
    print(fold_counter)

6
7
8
9


In [18]:
imagefolders

'/home/thanawit/twin-pseudo/cross_val_test/50%_CV/50%_Fold_9/unlabeled'

In [ ]:


result = pd.DataFrame(columns=["img_name", "conv_results", "swin_results","original_label"])

imagefolders = "/home/thanawit/twin-pseudo/cross_val_test/30%_CV/30%_Fold_0/unlabeled"


def apply_weak_augment(image):
    transform = A.Compose([
        
        A.HorizontalFlip(p=0.5,always_apply=True), #both
#         A.RandomGamma(p=0.5), #weak
#         A.RandomBrightnessContrast(p=0.5), #weak
#         A.AdvancedBlur(p=0.5), #weak
#         A.Rotate (limit=10, p=0.5),
    ])
    image = Image.open(image_path).convert('RGB')
    augmented_image = transform(image=np.array(image))['image']
    augmented_image = Image.fromarray(augmented_image)
    return augmented_image
    
    
    
#Swin Model

swin_model_path = 'outputs_all/conv_output_30%_Fold_0/checkpoint-2600/'
swin_model = AutoModelForImageClassification.from_pretrained(swin_model_path).to('cuda')
swin_image_processor = AutoImageProcessor.from_pretrained(swin_model_path)

#ConvNext Model
conv_model_path = 'outputs_all/conv_output_30%_Fold_0/checkpoint-2600/'
conv_model = AutoModelForImageClassification.from_pretrained(conv_model_path).to('cuda')
conv_image_processor = AutoImageProcessor.from_pretrained(conv_model_path)


for root, subfolders, files in os.walk(imagefolders):
    for filename in files:
        if filename.lower().endswith(image_formats):
            image_path = os.path.join(root, filename)
            subfolder_name = os.path.basename(root)

            #print(f"Subfolder Name: {subfolder_name}")
            #print(image_path)
            img_strong = apply_weak_augment(image_path)
            #img = Image.open(image_path)
            
            #CONNVEXT PART
            
            #read model
            conv_inputs = conv_image_processor(images=img_strong,return_tensors="pt").to('cuda')
            #run img
            conv_output = conv_model(**conv_inputs)
            conv_logits = conv_output.logits
            conv_class = conv_logits.argmax(-1).item()
            conv_classname = conv_model.config.id2label[conv_class]
            
            #SWIN PART
            
            img_weak = apply_weak_augment(image_path)
            
            swin_inputs = swin_image_processor(images=img_weak,return_tensors="pt").to('cuda')
            
            swin_output = swin_model(**swin_inputs)
            swin_logits = swin_output.logits
            swin_class = swin_logits.argmax(-1).item()
            swin_classname = swin_model.config.id2label[swin_class]
            #get class
            new_row = {
                "img_name": os.path.basename(image_path),
                "conv_results": conv_classname,
                "swin_results": swin_classname,
                "original_label":subfolder_name
            }
            
            result = pd.concat([result, pd.DataFrame([new_row])],ignore_index=True)
            #save 
            #break
            #img.show()
            #print("=" * 30)
        #print(filename)
        
basefolder_name = (os.path.dirname(imagefolders))
fold_no = os.path.dirname(os.path.dirname(imagefolders))

for index, row in result.iterrows():
    if row['conv_results'] == row['original_label'] or row['swin_results'] == row['original_label'] :
        temp = os.path.join(imagefolders, row['original_label'])
        temp2 = os.path.join( temp,row['img_name'] )
        copy_and_move_image(temp2,row['original_label'] ,os.path.join(basefolder_name, 'pseudo_labeled'  ))
        
# combine_folders([os.path.join(basefolder_name,'pseudo_labeled')],
#                 os.path.join(basefolder_name,'combine_pseudo_labeled') )


train_folder = os.path.join(basefolder_name,'train')
pseudo_folder = os.path.join(basefolder_name,'pseudo_labeled')
source_folders = []
source_folders.append(train_folder)
source_folders.append(pseudo_folder)
destination_folder = os.path.join(basefolder_name,'combine_pseudo_labeled')
                            
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Iterate through each source folder
for source_folder in source_folders:
    # Traverse through the source folder and its subdirectories to find all files and folders
    for root, dirs, files in os.walk(source_folder):
        # Create corresponding subdirectories in the destination folder
        dest_subfolder = os.path.join(destination_folder, os.path.relpath(root, source_folder))
        if not os.path.exists(dest_subfolder):
            os.makedirs(dest_subfolder)

        # Copy files to the destination folder
        for file in files:
            src_path = os.path.join(root, file)
            dest_path = os.path.join(dest_subfolder, file)
            shutil.copy(src_path, dest_path)


In [19]:
def apply_weak_augment(image):
    transform = A.Compose([
        A.Affine (scale=None, translate_percent=None, translate_px=None, rotate=None, shear=10, interpolation=1, mask_interpolation=0, cval=0, cval_mask=0, mode=0, fit_output=False, keep_ratio=False, rotate_method='largest_box', always_apply=False),
        A.RandomGamma(always_apply=True,p=1), #both
        A.RandomGamma(p=0.5), #weak
        A.RandomBrightnessContrast(p=0.5), #weak
        A.AdvancedBlur(p=0.5), #weak RandomRotate90
        A.Rotate (limit=10, p=0.5),
    ])
    image = Image.open(image_path).convert('RGB')
    augmented_image = transform(image=np.array(image))['image']
    augmented_image = Image.fromarray(augmented_image)
    return augmented_image
    
def apply_strong_augment(image):
    transform = A.Compose([
#         A.GridDropout (ratio=0.5, unit_size_min=2, unit_size_max=3, holes_number_x=None, holes_number_y=None, shift_x=0, shift_y=0, random_offset=False, fill_value=0, mask_fill_value=None, always_apply=False, p=0.2),
#         A.CLAHE (clip_limit=4.0, tile_grid_size=(8, 8), always_apply=True, p=1), # Heavy Contrast Limited Adaptive Histogram Equalization
#         A.Affine(), #heavy (shear + rotate)
#         A.RandomSunFlare(src_radius=200), #heavy
#         A.CoarseDropout (max_holes=8, max_height=8, max_width=8, min_holes=None, min_height=None, min_width=None, fill_value=0, mask_fill_value=None, always_apply=False, p=0.5),#heavy
#         A.RandomCrop(450,450), #80% of image size, probably both
#         A.GaussNoise (var_limit=(10.0, 50.0), mean=0, per_channel=True, always_apply=False, p=0.5),
        
        #A.RandomFog(), # heavy
    ])
    image = Image.open(image_path).convert('RGB')
    augmented_image = transform(image=np.array(image))['image']
    augmented_image = Image.fromarray(augmented_image)
    return augmented_image

In [42]:
image_formats = (".png", ".jpg", ".jpeg", ".gif", ".bmp")  # Add more supported formats if needed

result = pd.DataFrame(columns=["img_name", "conv_results", "swin_results","original_label"])

#30% Test
imagefolders = "/home/thanawit/twin-pseudo/cross_val_test/30%_CV/30%_Fold_0/unlabeled"
#Swin Model
swin_model_path = 'outputs_all/outputs_swin/checkpoint-16900/'
swin_model = AutoModelForImageClassification.from_pretrained(swin_model_path).to('cuda')
swin_image_processor = AutoImageProcessor.from_pretrained(swin_model_path)

#ConvNext Model
conv_model_path = 'outputs_all/outputs_convnext/checkpoint-16900/'
conv_model = AutoModelForImageClassification.from_pretrained(conv_model_path).to('cuda')
conv_image_processor = AutoImageProcessor.from_pretrained(conv_model_path)




for root, subfolders, files in os.walk(imagefolders):
    for filename in files:
        if filename.lower().endswith(image_formats):
            image_path = os.path.join(root, filename)
            subfolder_name = os.path.basename(root)

            #print(f"Subfolder Name: {subfolder_name}")
            #print(image_path)
            img_strong = apply_weak_augment(image_path)
            #img = Image.open(image_path)
            
            #CONNVEXT PART
            
            #read model
            conv_inputs = conv_image_processor(images=img_strong,return_tensors="pt").to('cuda')
            #run img
            conv_output = conv_model(**conv_inputs)
            conv_logits = conv_output.logits
            conv_class = conv_logits.argmax(-1).item()
            conv_classname = conv_model.config.id2label[conv_class]
            
            #SWIN PART
            
            img_weak = apply_weak_augment(image_path)
            
            swin_inputs = swin_image_processor(images=img_weak,return_tensors="pt").to('cuda')
            
            swin_output = swin_model(**swin_inputs)
            swin_logits = swin_output.logits
            swin_class = swin_logits.argmax(-1).item()
            swin_classname = swin_model.config.id2label[swin_class]
            #get class
            new_row = {
                "img_name": os.path.basename(image_path),
                "conv_results": conv_classname,
                "swin_results": swin_classname,
                "original_label":subfolder_name
            }
            
            result = pd.concat([result, pd.DataFrame([new_row])],ignore_index=True)
            #save 
            #break
            #img.show()
            #print("=" * 30)
        #print(filename)
#counter = print_rows_with_correct_label(result)
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def calculate_metrics(confusion_matrix):
    num_classes = len(confusion_matrix)
    precision_scores = []
    recall_scores = []
    f1_scores = []
    accuracy_scores = []
    auc_scores = []

    for i in range(num_classes):
        true_positives = confusion_matrix[i, i]
        false_positives = np.sum(confusion_matrix[:, i]) - true_positives
        false_negatives = np.sum(confusion_matrix[i, :]) - true_positives
        true_negatives = np.sum(confusion_matrix) - true_positives - false_positives - false_negatives

        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives + false_negatives)
        f1 = 2 * (precision * recall) / (precision + recall)
        accuracy = (true_positives + true_negatives) / np.sum(confusion_matrix[i, :])

        # Calculate AUC-ROC
        y_true = np.zeros(confusion_matrix.shape[0])
        y_true[i] = 1
        y_scores = confusion_matrix[:, i] / np.sum(confusion_matrix[:, i])
        auc = roc_auc_score(y_true, y_scores)

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
        accuracy_scores.append(accuracy)
        auc_scores.append(auc)

    average_precision = np.mean(precision_scores)
    average_recall = np.mean(recall_scores)
    average_f1 = np.mean(f1_scores)
    average_accuracy = np.mean(accuracy_scores)
    average_auc = np.mean(auc_scores)

    return precision_scores, recall_scores, f1_scores, accuracy_scores, auc_scores, average_precision, average_recall, average_f1, average_accuracy, average_auc

# Example usage
confusion_matrix = np.array([[10, 1, 2],
                             [3, 15, 1],
                             [0, 2, 12]])

precision_scores, recall_scores, f1_scores, accuracy_scores, auc_scores, average_precision, average_recall, average_f1, average_accuracy, average_auc = calculate_metrics(confusion_matrix)

for i in range(len(precision_scores)):
    print(f"Metrics for class {i}:")
    print(f"Precision: {precision_scores[i]}")
    print(f"Recall: {recall_scores[i]}")
    print(f"F1 Score: {f1_scores[i]}")
    print(f"Accuracy per class: {accuracy_scores[i]}")
    print(f"AUC-ROC: {auc_scores[i]}\n")

print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1 Score: {average_f1}")
print(f"Average Accuracy: {average_accuracy}")
print(f"Average AUC-ROC: {average_auc}")

KeyboardInterrupt: 

In [41]:

print("Total Record",len(result))
print("Correct Result from Convnet",print_rows_with_correct_conv(result))
print("Correct Result from Swinn",print_rows_with_correct_swin(result))
print("Correct Result from Any Correct Label",print_rows_with_correct_label(result))
print("Correct Result from Ensemble",print_rows_with_correct_ensemble(result))


Total Record 3802
Correct Result from Convnet 3471
Correct Result from Swinn 3378
Correct Result from Any Correct Label 3618
Correct Result from Ensemble 3231


In [3]:
def apply_weak_augment(image):
    transform = A.Compose([
        
        A.HorizontalFlip(p=0.5,always_apply=True), #both
#         A.RandomGamma(p=0.5), #weak
#         A.RandomBrightnessContrast(p=0.5), #weak
#         A.AdvancedBlur(p=0.5), #weak
#         A.Rotate (limit=10, p=0.5),
    ])
    image = Image.open(image_path).convert('RGB')
    augmented_image = transform(image=np.array(image))['image']
    augmented_image = Image.fromarray(augmented_image)
    return augmented_image
    